In [2]:
import pymongo
import logging
import logging.handlers
import datetime

# Connect to MongoDB
client = pymongo.MongoClient('mongodb://localhost:27017/')
db = client['acme_inc']

def create_sitegroup(sitegroup_name, sitegroup_type):
    collection = db['sdwan']

    # Create sitegroup document
    sitegroup = {
        'sitegroup_name': sitegroup_name,
        'sitegroup_type': sitegroup_type,
        'members': {
            'hubs': [],
            'spokes': []
        },
        'created_at': datetime.datetime.utcnow()
    }

    try:
        # Insert sitegroup document and get _id field
        result = collection.insert_one(sitegroup)
        document_id = result.inserted_id

        # Log success message with creation time and document id
        logging.info(f"Created SiteGroup with _id {document_id} at {sitegroup['created_at']}")

        # Send syslog message with creation time and document id
        syslog_logger.info(f"Created SiteGroup with _id {document_id} at {sitegroup['created_at']}")
    except Exception as e:
        # Log error message with creation time and exception message
        logging.error(f"Failed to create SiteGroup at {datetime.datetime.utcnow()}: {e}")

        # Send syslog message with creation time and exception message
        syslog_logger.error(f"Failed to create SiteGroup at {datetime.datetime.utcnow()}: {e}")
        raise e

    # Return _id field
    return str(document_id)


# Initialize logging
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

# Initialize syslog logger
syslog_logger = logging.getLogger('syslog')
syslog_logger.setLevel(logging.INFO)
syslog_handler = logging.handlers.SysLogHandler(address=('192.168.10.199', 3514))
syslog_logger.addHandler(syslog_handler)

# Call create_sitegroup function with dictionary input
sitegroup_data = {
    'sitegroup_name': 'SiteGroup1',
    'sitegroup_type': 'fullmesh'
}
document_id = create_sitegroup(**sitegroup_data)

# Close logging
logging.shutdown()


2023-04-02 11:06:16,217 - INFO - Created SiteGroup with _id 642999e86c1b5ba9537ec617 at 2023-04-02 15:06:16.183408
2023-04-02 11:06:16,219 - INFO - Created SiteGroup with _id 642999e86c1b5ba9537ec617 at 2023-04-02 15:06:16.183408
